In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #to ignore pandas warning
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *
import itertools


## Data Processing

### Applying One-Hot Encoding and Shuffling

In [2]:
col_names = ["Class", "a1", "a2", "a3", "a4", "a5", "a6", "ID"]
tr_data = pd.read_csv("../monk+s+problems/monks-1.train", sep =" ",  names = col_names)
test_data = pd.read_csv("../monk+s+problems/monks-1.test", sep =" ",  names = col_names)

tr_data = process_monk_data(tr_data)
test_data = process_monk_data(test_data)

print(tr_data)

     Class        ID  a1_1  a1_2  a1_3  a2_1  a2_2  a2_3  a3_1  a3_2  a4_1  \
0        1   data_42     1     0     0     1     0     0     0     1     0   
1        0  data_157     0     1     0     1     0     0     1     0     0   
2        0  data_143     1     0     0     0     0     1     0     1     0   
3        0   data_93     1     0     0     0     1     0     0     1     0   
4        1  data_197     0     1     0     0     1     0     1     0     1   
..     ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
119      1  data_212     0     1     0     0     1     0     1     0     0   
120      1  data_206     0     1     0     0     1     0     1     0     0   
121      1  data_414     0     0     1     0     0     1     0     1     1   
122      1  data_153     0     1     0     1     0     0     1     0     0   
123      0  data_144     1     0     0     0     0     1     0     1     0   

     a4_2  a4_3  a5_1  a5_2  a5_3  a5_4  a6_1  a6_2  
0       0

## Creating the Neural Network

In [3]:

'''net = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net.add_input_layer(n_inputs)

net.add_hidden_layer(n_inputs, 4)

net.add_output_layer(4, 1)
'''

'net = NeuralNetwork()\n\nn_inputs = tr_data.shape[1] - 2\n\nnet.add_input_layer(n_inputs)\n\nnet.add_hidden_layer(n_inputs, 4)\n\nnet.add_output_layer(4, 1)\n'

## Training

### Training with Mean Square Error

In [4]:
hyperparams = {"epochs": 500,
               "eta": 0.5,
               "clip_value": 1000,
               "hid_act_fun": "tanh",
               "out_act_fun": "sigmoid",
               "cost_fun": "mse",
               "mb" : 16,
               "momentum" : 0.01}


#losses = net.train(tr_data, hyperparams)  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

#plot_loss(losses, cost_fun = hyperparams["cost_fun"])

### Training with Binary Cross Entropy

In [5]:
'''
#RESET THE NETWORK!!!

losses = net.train(tr_data_X, tr_data_Y, epochs=1000,
                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

plot_loss(losses, cost_fun="b_ce")
'''

'\n#RESET THE NETWORK!!!\n\nlosses = net.train(tr_data_X, tr_data_Y, epochs=1000,\n                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE\n# print(net)\n# print(net.input_layer.weights)\n\nplot_loss(losses, cost_fun="b_ce")\n'

# Test Monk 1

In [6]:
#best_eta = net.k_fold(3, tr_data, hyperparams, "eta", [0.01, 0.05, 0.1,0.5, 0.8, 1])
#print(best_eta)

In [7]:
grid = {
    "eta" : [0.1, 0.2],
    "mb" : [32, 64],
    "momentum" : [0.01, 0.05, 0.1],
    "n_layers" : [2, 5],
    "n_neurons" : [10],
    "epochs" : [50],
    "clip_value" : [1000],
    "hid_act_fun" : ["tanh"],
    "out_act_fun" : ["sigmoid"],
    "cost_fun" : ["mse"],
    "ridge_lambda": [0.01],
    "lasso_lambda": [None],
    "linear_decay": [True],
    "eta0": [1],
    "max_steps": [100],
    "epochs_update": [2]}
    


tuple_search_space = list(itertools.product(grid["eta"], grid["mb"], grid["momentum"], grid["n_layers"], grid["n_neurons"], grid["epochs"],
                            grid["clip_value"], grid["hid_act_fun"], grid["out_act_fun"], grid["cost_fun"], grid["ridge_lambda"], grid["lasso_lambda"],
                            grid["linear_decay"], grid["eta0"], grid["max_steps"], grid["epochs_update"]))
dict_search_space = []

for conf in tuple_search_space:
    dict_search_space.append(dict(zip(grid.keys(),conf))) 

print(dict_search_space)
best_conf, min_err, all_configs = grid_search(2, tr_data, dict_search_space, n_inputs = tr_data.shape[1] - 2, n_outputs=1)

[{'eta': 0.1, 'mb': 32, 'momentum': 0.01, 'n_layers': 2, 'n_neurons': 10, 'epochs': 50, 'clip_value': 1000, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'mse', 'ridge_lambda': 0.01, 'lasso_lambda': None, 'linear_decay': True, 'eta0': 1, 'max_steps': 100, 'epochs_update': 2}, {'eta': 0.1, 'mb': 32, 'momentum': 0.01, 'n_layers': 5, 'n_neurons': 10, 'epochs': 50, 'clip_value': 1000, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'mse', 'ridge_lambda': 0.01, 'lasso_lambda': None, 'linear_decay': True, 'eta0': 1, 'max_steps': 100, 'epochs_update': 2}, {'eta': 0.1, 'mb': 32, 'momentum': 0.05, 'n_layers': 2, 'n_neurons': 10, 'epochs': 50, 'clip_value': 1000, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'mse', 'ridge_lambda': 0.01, 'lasso_lambda': None, 'linear_decay': True, 'eta0': 1, 'max_steps': 100, 'epochs_update': 2}, {'eta': 0.1, 'mb': 32, 'momentum': 0.05, 'n_layers': 5, 'n_neurons': 10, 'epochs': 50, 'clip_value': 1000, 'hid_act_fun': 'tan

In [8]:
print(all_configs)
print(min_err)
print(best_conf)

#0.012976971611538296

{frozenset({('eta0', 1), ('n_layers', 2), ('momentum', 0.01), ('n_neurons', 10), ('epochs', 50), ('clip_value', 1000), ('hid_act_fun', 'tanh'), ('epochs_update', 2), ('eta', 0.1), ('max_steps', 100), ('cost_fun', 'mse'), ('mb', 32), ('linear_decay', True), ('ridge_lambda', 0.01), ('lasso_lambda', None), ('out_act_fun', 'sigmoid')}): 0.05269793859259497, frozenset({('eta0', 1), ('epochs', 50), ('momentum', 0.01), ('n_neurons', 10), ('hid_act_fun', 'tanh'), ('n_layers', 5), ('clip_value', 1000), ('epochs_update', 2), ('eta', 0.1), ('max_steps', 100), ('cost_fun', 'mse'), ('mb', 32), ('linear_decay', True), ('ridge_lambda', 0.01), ('lasso_lambda', None), ('out_act_fun', 'sigmoid')}): 0.0645027838046187, frozenset({('eta0', 1), ('n_layers', 2), ('epochs', 50), ('n_neurons', 10), ('hid_act_fun', 'tanh'), ('clip_value', 1000), ('epochs_update', 2), ('eta', 0.1), ('max_steps', 100), ('cost_fun', 'mse'), ('mb', 32), ('ridge_lambda', 0.01), ('linear_decay', True), ('momentum', 0.05), ('lasso_la